<a href="https://www.kaggle.com/code/mh0386/medical-insurance-cost-prediction?scriptVersionId=120964541" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import statsmodels.api as sm

ModuleNotFoundError: No module named 'pandas'

# The Data

In [5]:
data = pd.read_csv('/kaggle/input/insurance/insurance.csv')
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


# Null Values

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
for col in data.columns:
    fig, ax = plt.subplots()
    plt.title(f'{col} histogram')
    sns.histplot(data=data, x=col, ax=ax)
    plt.show()

In [ ]:
data.smoker = le.fit_transform(data.smoker)
data.sex = le.fit_transform(data.sex)
data.region = le.fit_transform(data.region)
data

In [ ]:
data.info()

# Split the data to train and test data

In [ ]:
Y = data.charges
data.drop('charges', axis=1, inplace=True)
X = np.asarray(data)
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.20, random_state=42)

# Perform Data Scaling using Robust Scaler

In [ ]:
rs = RobustScaler()
X_Train = rs.fit_transform(X_Train)
X_Test = rs.fit_transform(X_Test)

# Linear Regression Model

In [ ]:
Model = linear_model.LinearRegression()
Model.fit(X_Train, Y_Train)

In [ ]:
Model.score(X_Train, Y_Train)

In [ ]:
Model.score(X_Test, Y_Test)

In [ ]:
Y_Pred = Model.predict(X_Test)
Y_Test_Pred = pd.DataFrame({"Y_Test": Y_Test, "Y_Pred": Y_Pred})
Y_Test_Pred.head()

In [ ]:
Y_Test_Pred = Y_Test_Pred.reset_index()

In [ ]:
plt.figure(figsize=(15, 8))
Y_Test_Pred = Y_Test_Pred.drop(["index"], axis=1)
plt.plot(Y_Test_Pred)
plt.legend(["Actual", "Predicted"])

In [ ]:
reg_score = r2_score(Y_Test, Y_Pred)
reg_score

In [ ]:
mse = mean_squared_error(Y_Test, Y_Pred)
print("mse: ", mse)
mape = mean_absolute_percentage_error(Y_Test, Y_Pred)
print("mape: ", mape)
mae = mean_absolute_error(Y_Test, Y_Pred)
print("mae: ", mae)
reg_mse = mean_squared_error(Y_Test, Y_Pred)
reg_rmse = np.sqrt(reg_mse)
print("reg_mse: ", reg_rmse)

In [ ]:
X2 = sm.add_constant(X_Train)
est = sm.OLS(Y_Train, X2)
est2 = est.fit()

In [ ]:
est2.summary()